In [1]:
from gensim.models import Word2Vec
from _DevNLTKPatentPriorArtFinder import _DevNLTKPatentPriorArtFinder as paf
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\mocka\mambaforge\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mocka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mocka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Will read the .json to a dataframe
jsonObject1= pd.io.json.read_json("JsonTTPatentsWithcitations.json")
#jsonObject1 = pd.io.json.read_json('/content/JsonTTPatentsWithcitations.json')
jsonObject1

,publication_number,abstract_en,citations,CPCs
0,US-2018343744-A1,A hybrid microelectronic substrate may be form...,"US-7294529-B2,US-7842541-B1,US-7935893-B2,US-9...","H05K1/183,H05K1/183,H05K1/183,H05K1/183,H05K1/..."
1,US-2019038230-A1,A composite monitoring apparatus includes a bi...,"US-6533184-B1,US-2011133919-A1,US-2011132136-A...","B60W2540/221,B60W2540/221,B60W2540/221,B60W254..."
2,US-2018254686-A1,A magnetic sensor includes a magnetic detectio...,"US-2003080442-A1,US-2010219822-A1,US-201120488...","G01R33/091,G01R33/091,G01R33/091,G01R33/091,G0..."
3,US-2017338526-A1,A battery pack according to an exemplary aspec...,"US-6621244-B1,US-2012065826-A1,US-2012225331-A...","H01M10/4207,H01M10/4207,H01M10/4207,H01M10/420..."
4,US-2017160890-A1,A terminal apparatus configured to provide a c...,"US-5848373-A,US-6321158-B1,US-2002130904-A1,US...","G06F3/04847,G06F3/04847,G06F3/04847,G06F3/0484..."
...,...,...,...,...
65,US-2020049406-A1,Methods and apparatuses for drying electronic ...,"US-9816757-B1,US-10240867-B2,US-2019219332-A1,...","F26B3/20,F26B3/20,F26B3/20,F26B5/04,F26B5/04,F..."
66,US-2017010186-A1,The present invention discloses a inspection d...,"US-3951563-A,US-3951563-A,US-3951563-A,US-3951...","G01N2291/2696,G01N2291/106,G01N29/24,G01N29/04..."
67,US-2019317829-A1,Computational methods and systems that proacti...,"US-2010083248-A1,US-2015308938-A1,US-201008324...","G06N7/08,G06N7/08,G06F11/3006,G06F11/3006,G06F..."
68,US-2019089374-A1,A method for performing low-density parity che...,"US-9235467-B2,US-9235467-B2,US-9235467-B2,US-9...","H03M13/1125,H03M13/3707,H03M13/1125,H03M13/110..."


In [4]:
myPaf = paf()
dframe = myPaf.init(jsonObject1,"publication_number", "abstract_en")

In [5]:
dframe

,publication_number,abstract_en,citations,CPCs,Tokens,BagOfWords,TF-IDF
0,US-2018343744-A1,A hybrid microelectronic substrate may be form...,"US-7294529-B2,US-7842541-B1,US-7935893-B2,US-9...","H05K1/183,H05K1/183,H05K1/183,H05K1/183,H05K1/...","[hybrid, microelectronic, substrate, may, form...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,US-2019038230-A1,A composite monitoring apparatus includes a bi...,"US-6533184-B1,US-2011133919-A1,US-2011132136-A...","B60W2540/221,B60W2540/221,B60W2540/221,B60W254...","[composite, monitoring, apparatus, includes, b...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,US-2018254686-A1,A magnetic sensor includes a magnetic detectio...,"US-2003080442-A1,US-2010219822-A1,US-201120488...","G01R33/091,G01R33/091,G01R33/091,G01R33/091,G0...","[magnetic, sensor, includes, magnetic, detecti...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,US-2017338526-A1,A battery pack according to an exemplary aspec...,"US-6621244-B1,US-2012065826-A1,US-2012225331-A...","H01M10/4207,H01M10/4207,H01M10/4207,H01M10/420...","[battery, pack, according, exemplary, aspect, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 140, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,US-2017160890-A1,A terminal apparatus configured to provide a c...,"US-5848373-A,US-6321158-B1,US-2002130904-A1,US...","G06F3/04847,G06F3/04847,G06F3/04847,G06F3/0484...","[terminal, apparatus, configured, provide, cha...","[1105, 0, 0, 0, 0, 0, 0, 0, 0, 1105, 0, 0, 0, ...","[0.0523973747783371, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...
65,US-2020049406-A1,Methods and apparatuses for drying electronic ...,"US-9816757-B1,US-10240867-B2,US-2019219332-A1,...","F26B3/20,F26B3/20,F26B3/20,F26B5/04,F26B5/04,F...","[methods, apparatuses, drying, electronic, dev...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
66,US-2017010186-A1,The present invention discloses a inspection d...,"US-3951563-A,US-3951563-A,US-3951563-A,US-3951...","G01N2291/2696,G01N2291/106,G01N29/24,G01N29/04...","[present, invention, discloses, inspection, de...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
67,US-2019317829-A1,Computational methods and systems that proacti...,"US-2010083248-A1,US-2015308938-A1,US-201008324...","G06N7/08,G06N7/08,G06F11/3006,G06F11/3006,G06F...","[computational, methods, systems, proactively,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 108, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
68,US-2019089374-A1,A method for performing low-density parity che...,"US-9235467-B2,US-9235467-B2,US-9235467-B2,US-9...","H03M13/1125,H03M13/3707,H03M13/1125,H03M13/110...","[method, performing, lowdensity, parity, check...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [7]:
print(dframe.at[0,'BagOfWords'])
myPaf.word_count_matrix

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 364, 364, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 364, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 364, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 364, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36

,_num_,ability,absorbent,access,accessing,accessories,accessory,accommodate,accordance,according,...,wiring,within,without,workpiece,wound,xyscan,yawing,zdevice,zones,zscan
0,0,0,0,0,0,0,0,0,0,0,...,0,364,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,140,...,0,0,0,0,0,0,0,0,0,0
4,1105,0,0,0,0,0,0,0,0,1105,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0,0,0,0,0,0,0,0,0,0,...,0,108,0,0,0,0,0,0,0,0
66,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68,0,0,0,0,0,0,0,0,0,0,...,0,0,40,0,0,0,0,0,0,0


For some reason in the BagOfWords function, it's not showing how many times the word is in the abstract correctly, i.e. 364 instead of 1, in this particular example. Or 1105 instead of 1 in Patent 4.

Anyways, the rest of the demonstration only needs the tokens, which is accurate info.

In [17]:
#First model is created with the tokens in sentences
sentences= dframe['Tokens']
model = Word2Vec(sentences)

In [18]:
#Just an example to show that it's working
print(model.wv.most_similar(positive=['device']))

[('output', 0.5668588876724243), ('coupling', 0.5625364184379578), ('content', 0.5518147945404053), ('displayed', 0.5491979718208313), ('detects', 0.541864812374115), ('also', 0.5363319516181946), ('reliable', 0.5307567119598389), ('causing', 0.5193239450454712), ('alters', 0.48386865854263306), ('stable', 0.47696295380592346)]


In [26]:
#Tokenizing all the citations and displaying it's header/column in the next codeblock
def tokenizeCitations(citationsString):
  string1 = citationsString.replace(',',' ')
  tokenized = word_tokenize(string1)
  return list(set(tokenized))

def allTokenizeCitations(dataframe):
  dataframe['TokenizedCitations'] = dataframe['citations'].apply(tokenizeCitations)

In [27]:
allTokenizeCitations(dframe)
dframe

,publication_number,abstract_en,citations,CPCs,Tokens,BagOfWords,TF-IDF,TokenizedCitations
0,US-2018343744-A1,A hybrid microelectronic substrate may be form...,"US-7294529-B2,US-7842541-B1,US-7935893-B2,US-9...","H05K1/183,H05K1/183,H05K1/183,H05K1/183,H05K1/...","[hybrid, microelectronic, substrate, may, form...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-7294529-B2, US-2019057915-A1, US-7935893-B..."
1,US-2019038230-A1,A composite monitoring apparatus includes a bi...,"US-6533184-B1,US-2011133919-A1,US-2011132136-A...","B60W2540/221,B60W2540/221,B60W2540/221,B60W254...","[composite, monitoring, apparatus, includes, b...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-2011132136-A1, US-2013070043-A1, US-201807..."
2,US-2018254686-A1,A magnetic sensor includes a magnetic detectio...,"US-2003080442-A1,US-2010219822-A1,US-201120488...","G01R33/091,G01R33/091,G01R33/091,G01R33/091,G0...","[magnetic, sensor, includes, magnetic, detecti...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-2016282144-A1, US-2011204887-A1, US-201021..."
3,US-2017338526-A1,A battery pack according to an exemplary aspec...,"US-6621244-B1,US-2012065826-A1,US-2012225331-A...","H01M10/4207,H01M10/4207,H01M10/4207,H01M10/420...","[battery, pack, according, exemplary, aspect, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 140, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-2012065826-A1, US-2016329545-A1, US-201302..."
4,US-2017160890-A1,A terminal apparatus configured to provide a c...,"US-5848373-A,US-6321158-B1,US-2002130904-A1,US...","G06F3/04847,G06F3/04847,G06F3/04847,G06F3/0484...","[terminal, apparatus, configured, provide, cha...","[1105, 0, 0, 0, 0, 0, 0, 0, 0, 1105, 0, 0, 0, ...","[0.0523973747783371, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-2002130904-A1, US-2012226757-A1, US-201110..."
...,...,...,...,...,...,...,...,...
65,US-2020049406-A1,Methods and apparatuses for drying electronic ...,"US-9816757-B1,US-10240867-B2,US-2019219332-A1,...","F26B3/20,F26B3/20,F26B3/20,F26B5/04,F26B5/04,F...","[methods, apparatuses, drying, electronic, dev...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-10240867-B2, US-9816757-B1, US-2019219332-A1]"
66,US-2017010186-A1,The present invention discloses a inspection d...,"US-3951563-A,US-3951563-A,US-3951563-A,US-3951...","G01N2291/2696,G01N2291/106,G01N29/24,G01N29/04...","[present, invention, discloses, inspection, de...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[US-3951563-A]
67,US-2019317829-A1,Computational methods and systems that proacti...,"US-2010083248-A1,US-2015308938-A1,US-201008324...","G06N7/08,G06N7/08,G06F11/3006,G06F11/3006,G06F...","[computational, methods, systems, proactively,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 108, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-2010083248-A1, US-2015308938-A1]"
68,US-2019089374-A1,A method for performing low-density parity che...,"US-9235467-B2,US-9235467-B2,US-9235467-B2,US-9...","H03M13/1125,H03M13/3707,H03M13/1125,H03M13/110...","[method, performing, lowdensity, parity, check...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[US-9235467-B2]


In [28]:
#Showing that both sentences and sentences2 are tokenized correctly
sentences

0     [hybrid, microelectronic, substrate, may, form...
1     [composite, monitoring, apparatus, includes, b...
2     [magnetic, sensor, includes, magnetic, detecti...
3     [battery, pack, according, exemplary, aspect, ...
4     [terminal, apparatus, configured, provide, cha...
                            ...                        
65    [methods, apparatuses, drying, electronic, dev...
66    [present, invention, discloses, inspection, de...
67    [computational, methods, systems, proactively,...
68    [method, performing, lowdensity, parity, check...
69    [highly, robust, communication, system, capabl...
Name: Tokens, Length: 70, dtype: object

In [29]:
sentences2 = dframe['TokenizedCitations']
sentences2

0     [US-7294529-B2, US-2019057915-A1, US-7935893-B...
1     [US-2011132136-A1, US-2013070043-A1, US-201807...
2     [US-2016282144-A1, US-2011204887-A1, US-201021...
3     [US-2012065826-A1, US-2016329545-A1, US-201302...
4     [US-2002130904-A1, US-2012226757-A1, US-201110...
                            ...                        
65    [US-10240867-B2, US-9816757-B1, US-2019219332-A1]
66                                       [US-3951563-A]
67                 [US-2010083248-A1, US-2015308938-A1]
68                                      [US-9235467-B2]
69    [US-2012302160-A1, US-2006009262-A1, US-201828...
Name: TokenizedCitations, Length: 70, dtype: object

In [36]:
# Created the 2nd model based on the citations, and an example
model2 = Word2Vec(sentences2, min_count=1)
model2.wv.most_similar(positive=['US-7294529-B2'])

[('US-2013267153-A1', 0.33723217248916626),
 ('US-2014076795-A1', 0.3185223340988159),
 ('US-2010033371-A1', 0.2665988504886627),
 ('US-5783077-A', 0.24466368556022644),
 ('US-5897011-A', 0.2397015392780304),
 ('US-2010164733-A1', 0.2306135594844818),
 ('US-2016110965-A1', 0.22854644060134888),
 ('US-9066619-B2', 0.2245655655860901),
 ('US-2012167154-A1', 0.22268284857273102),
 ('US-2012290950-A1', 0.21999993920326233)]

In [ ]:
#Finally the code for all patents that we have, to combine both the abstract and the citations in one document into a document Vector.
#Disclaimer - hasnt been tested yet

vecs= []

for (doc,citationList) in zip(sentences,sentences2):
  sum=np.empty(100)
  for word in doc:
    try:
      sum += model.wv[word]
    except:
      pass
  for citation in citationList:
    try:
      sum += model2.wv[citation]
    except:
      pass
  vecs.append(sum)